# DenseNet 
Todo: Discussion

In [ ]:
import os
import pathlib
from glob import glob
from typing import Union
import pandas as pd
import warnings

import tensorflow as tf
import tensorflow_models as tfm
from bcd.model.repo import ModelRepo
from bcd.model.factory import DenseNetFactory
from bcd.model.transfer import FineTuner
from bcd.model.callback import Historian
pd.set_option('display.max_rows',999)


## Configuration

In [ ]:
# Data params
batch_size = 32
input_shape = (224,224,3)
output_shape = 1
train_dir = pathlib.Path("data/image/1_final/training/training/").with_suffix('')
test_dir = pathlib.Path("data/image/1_final/test/test/").with_suffix('')

# Training Params
initial_epochs = 100  # Number of epochs to train for feature extraction
fine_tune_epochs = 50  # Number of epochs for each fine tune session
initial_learning_rate = 0.0001  # Base learning rate for the Adam optimizer 
loss = "binary_crossentropy"
activation = "sigmoid"

# Early stop callback 
min_delta = 0.0001
monitor = "val_loss"  # Monitor validation loss for early stopping
patience = 3  # The number of consecutive epochs for which lack of improvement is tolerated 
restore_best_weights = True  # Returns the best weights rather than the weights at the last epoch.


# ModelCheckpoint Callback parameters
location = "models/"
mode = "auto"
save_weights_only = False
save_best_only = True
save_freq = "epoch"
verbose = 1

# Historian parameters
historian_filepath = "models/densenet/densenet_history.pkl"

# Model Parameters
name = "densenet"
metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
model_factory = DenseNetFactory
force = False  # Whether to retrain if the model and weights already exist from a prior training session.
base_model_layer = 5 # Layer of the DenseNet base model

## Dependencies
Several dependencies will be used throughout this notebook, including:
- Early Stop Callback
- Learning Rate Warmup
- Model Repository
- DenseNet Model Factory
- Historian Callback

We'll make those objects available here.

In [ ]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor=monitor, 
                                                       min_delta=min_delta,
                                                       patience=patience, 
                                                       restore_best_weights=restore_best_weights)
learning_rate_warmup = tfm.optimization.LinearWarmup(after_warmup_lr_sched=0.0001,
                                                     warmup_steps=1000,
                                                     warmup_learning_rate=0.00001)
repo = ModelRepo(location=location)
factory = model_factory()
historian = Historian(name=name)

## Load Data

In [ ]:
# Training DataSet (10%)
train_ds_10 = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    seed=123,
    batch_size=batch_size)

# Validation DataSet (10%)
val_ds_10 = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    seed=123,
    batch_size=batch_size)

# Test Set
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True)

## Feature Extraction
If the model already exists, obtain it from the repository. Otherwise, create the model and perform feature extraction.

In [ ]:
stage = "feature_extraction"
if force or not repo.exists(name=name, stage=stage):
    model = factory.create(input_shape=input_shape, 
                            output_shape=output_shape, 
                            learning_rate=initial_learning_rate, 
                            trainable=False, 
                            loss=loss, 
                            activation=activation, 
                            metrics=metrics)
    # Delete existing checkpoints
    repo.remove(name=name, stage=stage)
    # Summarize the model
    model.summary()
    # Obtain a checkpoint callback from the model repository 
    checkpoint_callback = repo.create_callback(name=name, stage=stage)
    # Set the session on the historian to 0 for feature extraction
    historian.on_session_begin(session=0)
    # Fit the model with callbacks
    history = model.fit(train_ds_10, epochs=initial_epochs, validation_data=val_ds_10, callbacks=[checkpoint_callback, early_stop_callback, historian])        
    
else:
    # Obtain the model from the repository
    model = repo.get(name=name, stage=stage)
    # Obtain the historian
    historian = historian.load(historian_filepath)    
    # Summarize the model
    model.summary()




    

Feature extraction appears to have converged in fourteen epochs. Our early stopping callback stopped training when validation loss didn't improve in 3 epochs. Training and validation accuracy of 55% and 50% respectively, alludes to the considerable difference between source and target datasets. Gradually fine tuning the network on the CBIS-DDSM dataset will allow the network to adapt to the features and characteristics of our dataset.

The next cell will iteratively run a series of 10 fine tuning sessions in which the DenseNet model is gradually unfrozen in 10 increments. Each increment increases the number of layers to be thawed logarithmically from 1 layer to 707 layers of the DenseNet model. Learning rates will be decayed logarithmically from 1e-4 to 1e-10 to mitigate catastrophic forgetting within the network. 

Excellent. We can now move on to the fine tuning stage.

## Fine Tuning

In [14]:
ft = FineTuner(name=name, 
               train_ds=train_ds_10, 
               validation_ds=val_ds_10, 
               repo=repo,
               metrics=metrics,
               initial_learning_rate=1e-3,
               final_learning_rate=1e-6,
               callbacks=[early_stop_callback],
               )
ft.tune(model=model, historian=historian, base_model_layer=base_model_layer, force=force)

Fine tuning has completed. Let's check the learning curves.

In [ ]:
# Plot the historian
historian.plot_learning_curves()

## Evaluation
Let's evaluate generalization performance on the test set. 

In [ ]:
results = model.evaluate(test_ds, batch_size=batch_size)
print("Test Loss, Test Accuracy: ", results)